#### Definition of the model

* finding the optimal parameters
* finding the optimal equations for mu and qs
* applying **Latin Hypercube Sampling** for generating parameter sets
* calculating the **RMSE** of model and experiment

#### Code

In [1]:
import pandas as pd
import numpy as np
import yaml
from sklearn.metrics import mean_squared_error
from C_functions_opti import get_LHS_samples, model_optimization, model_optimization2, plot_estimation, model_optimization3

In [2]:
# Load experimental data
df_exp = pd.read_csv('data/data_combined.csv')
biomass_exp = df_exp['Biomass [g/L]']
substrate_exp = df_exp['Glucose [g/L]']

In [3]:
# Load parameters from YAML file
with open('config/parameters.yml', 'r') as file:
    param = yaml.safe_load(file)

The growth rate and the substrate uptake rate are dependend of each other - meaning that if we define a equation for **mu**, then **qs** is defined by mu with $qs=mu/Yxs$. Contrarily, when **qs** is defined by an ODE, then **mu** is calculated with $mu = qs * Yxs$.

In [4]:
mu0 = lambda mu_max, c_glucose, Ks: mu_max * c_glucose / (c_glucose + Ks) # -- MONOD
mu1 = lambda mu_max, c_biomass, X_max: mu_max * (1 - (c_biomass/ X_max)) # -- LOGISTIC
mu2 = lambda mu_max, c_glucose, Ks, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks)) * (1 - (c_biomass/ X_max)) # -- MONOD + LOGISTIC
mu3 = lambda mu_max, c_glucose, Ks, Ki, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks + (c_glucose**2/ Ki))) * (1 - (c_biomass/ X_max)) # -- MONOD + LOGISTIC + INHIBITION
mu4 = lambda qs, Yxs: qs * Yxs # for changing qs

In [5]:
qs0 = lambda qs_max, c_glucose, Ks_qs: qs_max * c_glucose / (Ks_qs + c_glucose) # -- MONOD
qs1 = lambda qs_max, c_glucose, Ks_qs, Ki, glu_met: qs_max * c_glucose / (Ks_qs + c_glucose) * (Ki / (Ki + glu_met)) # -- MONOD + NON COMPETITIVE INHIBITION
qs2 = lambda qs_max, c_glucose, Ks_qs, c_biomass, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(c_biomass * lag))) # -- MONOD + METABOLIZED GLU
qs3 = lambda mu, Yxs: mu / Yxs # for changing mu

In [6]:
# Saving all equations in each list
mu_all=[mu0, mu1, mu2, mu3, mu4]
qs_all=[qs0, qs1, qs2, qs3]

In [7]:
# Root mean squared error is the objective function
def objective_function(parameters, mu_eq, num_mu, qs_eq, num_qs):
    # Solve the model using the optimal parameters
    time_pred, biomass_pred, substrate_pred, volume_pred = model_optimization(param, parameters, mu_eq, num_mu, qs_eq, num_qs)  # Solve the model using the current parameters
    biomass = pd.concat([biomass_exp, pd.Series(biomass_pred)], axis=1, keys=['biomass_exp', 'biomass_pred']).dropna()
    biomass_exp_ = biomass['biomass_exp'].values
    biomass_pred_ = biomass['biomass_pred'].values
    mse_x = mean_squared_error(biomass_exp_, biomass_pred_)  # Calculate mean squared error for biomass

    glucose = pd.concat([substrate_exp, pd.Series(substrate_pred)], axis=1, keys=['substrate_exp', 'substrate_pred']).dropna()
    substrate_exp_ = glucose['substrate_exp'].values
    substrate_pred_ = glucose['substrate_pred'].values
    mse_s = mean_squared_error(substrate_exp_, substrate_pred_)  # Calculate mean squared error for substrate
    
    # Calculate the combined rmse
    mse = (mse_x + mse_s)/2
    rmse = np.sqrt(mse)  # Calculate root mean squared error
    return rmse, time_pred, biomass_pred, substrate_pred, volume_pred

In [8]:
# Root mean squared error is the objective function
def objective_function2(parameters, mu_eq, num_mu, qs_eq, num_qs):
    # Solve the model using the optimal parameters
    time_pred, biomass_pred, substrate_pred, volume_pred = model_optimization2(param, parameters, mu_eq, num_mu, qs_eq, num_qs)  # Solve the model using the current parameters
    biomass = pd.concat([biomass_exp, pd.Series(biomass_pred)], axis=1, keys=['biomass_exp', 'biomass_pred']).dropna()
    biomass_exp_ = biomass['biomass_exp'].values
    biomass_pred_ = biomass['biomass_pred'].values
    mse_x = mean_squared_error(biomass_exp_, biomass_pred_)  # Calculate mean squared error for biomass

    glucose = pd.concat([substrate_exp, pd.Series(substrate_pred)], axis=1, keys=['substrate_exp', 'substrate_pred']).dropna()
    substrate_exp_ = glucose['substrate_exp'].values
    substrate_pred_ = glucose['substrate_pred'].values
    mse_s = mean_squared_error(substrate_exp_, substrate_pred_)  # Calculate mean squared error for substrate
    
    # Calculate the combined rmse
    mse = (mse_x + mse_s)/2
    rmse = np.sqrt(mse)  # Calculate root mean squared error
    return rmse, time_pred, biomass_pred, substrate_pred, volume_pred

In [9]:
'''
## Version 1: Define all parameters manually in the config/parameters.yml file

# Extract the parameter combination and the number of the set in order to save it in the correct folder
est_mu_max = param['est_mu_max'] # ['set0']

# -------------------------------

## Version 2: Apply LH sampling on all parameters

# Set the number of samples and parameters
num_samples = 50
num_parameters = 9

# Define the ranges for each parameter
parameter_bounds = [
    [0.01, 0.6],    # Range for parameter 1 mu_max
    [18, 25],       # Range for parameter 2 X_max
    [0.1, 10.0],    # Range for parameter 3 - Ks
    [0.1, 10.0],    # Range for parameter 4 - Ks_qs
    [6.0, 8.0],     # Range for parameter 5 - Ki
    [0.2, 0.6],     # Range for parameter 6 - Yxs
    [0.5, 1.5],     # Range for parameter 7 - qs_max
    [0.05, 0.5],   # Range for parameter 8 - m_s
    [0.001, 0.02],  # Range for parameter 9 - lag
]
'''

"\n## Version 1: Define all parameters manually in the config/parameters.yml file\n\n# Extract the parameter combination and the number of the set in order to save it in the correct folder\nest_mu_max = param['est_mu_max'] # ['set0']\n\n# -------------------------------\n\n## Version 2: Apply LH sampling on all parameters\n\n# Set the number of samples and parameters\nnum_samples = 50\nnum_parameters = 9\n\n# Define the ranges for each parameter\nparameter_bounds = [\n    [0.01, 0.6],    # Range for parameter 1 mu_max\n    [18, 25],       # Range for parameter 2 X_max\n    [0.1, 10.0],    # Range for parameter 3 - Ks\n    [0.1, 10.0],    # Range for parameter 4 - Ks_qs\n    [6.0, 8.0],     # Range for parameter 5 - Ki\n    [0.2, 0.6],     # Range for parameter 6 - Yxs\n    [0.5, 1.5],     # Range for parameter 7 - qs_max\n    [0.05, 0.5],   # Range for parameter 8 - m_s\n    [0.001, 0.02],  # Range for parameter 9 - lag\n]\n"

#### First approach

In [10]:
# Set the number of samples and parameters
num_samples = 100
num_parameters = 9

# Define the ranges for each parameter
parameter_bounds = [
    [0.2, 0.4],    # Range for parameter 1 mu_max
    [17, 23],       # Range for parameter 2 X_max
    [0.1, 20.0],    # Range for parameter 3 - Ks
    [0.1, 20.0],    # Range for parameter 4 - Ks_qs
    [0.1, 20.0],     # Range for parameter 5 - Ki
    [0.3, 0.5],     # Range for parameter 6 - Yxs
    [0.1, 1.5],     # Range for parameter 7 - qs_max
    [0.0, 0.2],   # Range for parameter 8 - m_s
    [0.001, 1.0],  # Range for parameter 9 - lag
]

In [11]:
LHS_samples = get_LHS_samples(num_samples, num_parameters, parameter_bounds)
LHS_samples.shape

(100, 9)

In [12]:
df_all_sets = pd.DataFrame(columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
for set_num in range(LHS_samples.shape[0]):
    # Save all parameters and equations and the RMSE in a dataframe
    ## with the beginning of one set a new rmse_overview will be created
    rmse_one_set = []
    #key = f'set{set_num}' ; init_p = est_mu_max[key]
    init_p = list(LHS_samples[set_num, :])
    for i in range(len(mu_all)):
        # Define the combination of equations
        mu_eq = mu_all[i]; num_mu = i

        # mu0 to mu3 are combined with qs3 while for mu4 all qs except for qs3 are combined
        if i != 4:
            j = 3
            qs_eq = qs_all[j]; num_qs = j   

            # Make the predictions and calculate the error
            rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function(init_p, mu_eq, num_mu, qs_eq, num_qs)
            # save the parameters in a dataframe
            append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], init_p[4], init_p[5], init_p[6], init_p[7], init_p[8], round(rmse, 3)]
            rmse_one_set.append(append_list)

            # Make a plot and save it
            title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
            plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

            if rmse <= float(8):
                plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)
        
        elif i == 4:
            for j in range(3):
                qs_eq = qs_all[j]; num_qs = j   

                # Make the predictions and calculate the error
                rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function(init_p, mu_eq, num_mu, qs_eq, num_qs)
                # save the parameters in a dataframe
                append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], init_p[4], init_p[5], init_p[6], init_p[7], init_p[8], round(rmse, 3)]
                rmse_one_set.append(append_list)

                # Make a plot and save it
                title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
                plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

                if rmse <= float(8):
                    plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)

    # save the parameters of one set
    df_1set = pd.DataFrame(rmse_one_set, columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
    df_all_sets = pd.concat([df_all_sets,df_1set], axis=0, ignore_index=True)


df_all_sets.sort_values(by=['rmse'], ascending=True, inplace=True)
df_all_sets.to_csv(f'data/estimation/new_eq/data.csv')

df_all_sets.head(10)

,set,mu,qs,mu_max,X_max,Ks,Ks_qs,Ki,Yxs,qs_max,m_s,lag,rmse
394,56,2,3,0.375939,22.793818,3.575794,7.434359,5.929804,0.319374,0.807485,0.174286,0.551430,4.736
344,49,1,3,0.267825,22.061614,0.955530,2.174792,19.788262,0.324973,0.906631,0.188173,0.037222,5.294
309,44,1,3,0.283335,20.946448,0.417189,13.306026,17.232592,0.346334,0.868763,0.192866,0.060520,5.969
310,44,2,3,0.283335,20.946448,0.417189,13.306026,17.232592,0.346334,0.868763,0.192866,0.060520,6.484
43,6,1,3,0.355169,22.503048,9.102021,10.384339,6.279856,0.331876,1.180289,0.178892,0.177053,6.546
345,49,2,3,0.267825,22.061614,0.955530,2.174792,19.788262,0.324973,0.906631,0.188173,0.037222,6.644
393,56,1,3,0.375939,22.793818,3.575794,7.434359,5.929804,0.319374,0.807485,0.174286,0.551430,6.725
589,84,1,3,0.328302,19.957961,10.110884,13.885845,13.000297,0.382025,0.720380,0.196445,0.480148,7.378
183,26,1,3,0.275564,21.074165,4.592128,10.471263,9.503887,0.362235,0.973956,0.168017,0.376286,7.388
323,46,1,3,0.281174,18.481144,14.916234,1.629317,3.894219,0.339440,0.327416,0.180761,0.156053,8.236


In [13]:
df_LHS = pd.read_csv('data/estimation/LHS_sampling/data.csv')

#### Second approach

In [9]:
# Set the number of samples and parameters
num_samples = 100
num_parameters = 4

# Define the ranges for each parameter
parameter_bounds = [
    [0.2, 0.5], # mu_max
    [15, 25], # X_max
    [0.3, 0.5], # Yxs
    [0.01, 1.5] # m_s
]

LHS_samples = get_LHS_samples(num_samples, num_parameters, parameter_bounds)
LHS_samples.shape

(100, 4)

In [10]:
df_all_sets = pd.DataFrame(columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Yxs', 'm_s', 'rmse'])
for set_num in range(LHS_samples.shape[0]):
    # Save all parameters and equations and the RMSE in a dataframe
    ## with the beginning of one set a new rmse_overview will be created
    rmse_one_set = []
    #key = f'set{set_num}' ; init_p = est_mu_max[key]
    init_p = list(LHS_samples[set_num, :])
    
    i = 1; j = 3
    mu_eq = mu_all[i]; num_mu = i
    qs_eq = qs_all[j]; num_qs = j   

    # Make the predictions and calculate the error
    rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function2(init_p, mu_eq, num_mu, qs_eq, num_qs)
    # save the parameters in a dataframe
    append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], round(rmse, 3)]
    rmse_one_set.append(append_list)

    # Make a plot and save it
    title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
    plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

    if rmse <= float(6):
        plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)
    
    # save the parameters of one set
    df_1set = pd.DataFrame(rmse_one_set, columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Yxs', 'm_s', 'rmse'])
    df_all_sets = pd.concat([df_all_sets,df_1set], axis=0, ignore_index=True)


df_all_sets.sort_values(by=['rmse'], ascending=True, inplace=True)
df_all_sets.to_csv(f'data/estimation/2906_1/data_2906.csv')

df_all_sets.head(10)

,set,mu,qs,mu_max,X_max,Yxs,m_s,rmse
14,14,1,3,0.208841,20.157689,0.356392,0.379035,3.031
94,94,1,3,0.416085,15.261961,0.398926,0.363901,4.002
35,35,1,3,0.290946,20.496981,0.393092,0.331745,5.604
67,67,1,3,0.393152,18.920547,0.321152,0.233610,5.826
56,56,1,3,0.340263,21.969029,0.416104,0.209673,6.254
27,27,1,3,0.235071,15.765922,0.304430,0.274615,6.388
21,21,1,3,0.316678,16.061754,0.482967,0.514276,6.545
79,79,1,3,0.262088,24.010284,0.313227,0.126599,6.603
74,74,1,3,0.258713,21.533397,0.479812,0.222213,6.625
31,31,1,3,0.409120,17.906860,0.442952,0.256787,6.897


#### 3rd Approach: 4, 1

In [ ]:
# Root mean squared error is the objective function
def objective_function3(parameters, mu_eq, num_mu, qs_eq, num_qs):
    # Solve the model using the optimal parameters
    time_pred, biomass_pred, substrate_pred, volume_pred = model_optimization3(param, parameters, mu_eq, num_mu, qs_eq, num_qs)  # Solve the model using the current parameters
    biomass = pd.concat([biomass_exp, pd.Series(biomass_pred)], axis=1, keys=['biomass_exp', 'biomass_pred']).dropna()
    biomass_exp_ = biomass['biomass_exp'].values
    biomass_pred_ = biomass['biomass_pred'].values
    mse_x = mean_squared_error(biomass_exp_, biomass_pred_)  # Calculate mean squared error for biomass

    glucose = pd.concat([substrate_exp, pd.Series(substrate_pred)], axis=1, keys=['substrate_exp', 'substrate_pred']).dropna()
    substrate_exp_ = glucose['substrate_exp'].values
    substrate_pred_ = glucose['substrate_pred'].values
    mse_s = mean_squared_error(substrate_exp_, substrate_pred_)  # Calculate mean squared error for substrate
    
    # Calculate the combined rmse
    mse = (mse_x + mse_s)/2
    rmse = np.sqrt(mse)  # Calculate root mean squared error
    return rmse, time_pred, biomass_pred, substrate_pred, volume_pred

In [ ]:
# Set the number of samples and parameters
num_samples = 100
num_parameters = 5

# Define the ranges for each parameter
parameter_bounds = [
    [0.2, 0.5], # qs_max
    [15, 25], # Ks_qs
    [0.3, 0.5], # Ki
    [0.3, 0.5], # Yxs
    [0.01, 1.5] # m_s
]

LHS_samples = get_LHS_samples(num_samples, num_parameters, parameter_bounds)
LHS_samples.shape

In [ ]:
df_all_sets = pd.DataFrame(columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Yxs', 'm_s', 'rmse'])
for set_num in range(LHS_samples.shape[0]):
    # Save all parameters and equations and the RMSE in a dataframe
    ## with the beginning of one set a new rmse_overview will be created
    rmse_one_set = []
    #key = f'set{set_num}' ; init_p = est_mu_max[key]
    init_p = list(LHS_samples[set_num, :])
    
    i = 1; j = 3
    mu_eq = mu_all[i]; num_mu = i
    qs_eq = qs_all[j]; num_qs = j   

    # Make the predictions and calculate the error
    rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function2(init_p, mu_eq, num_mu, qs_eq, num_qs)
    # save the parameters in a dataframe
    append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], round(rmse, 3)]
    rmse_one_set.append(append_list)

    # Make a plot and save it
    title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
    plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

    if rmse <= float(6):
        plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)
    
    # save the parameters of one set
    df_1set = pd.DataFrame(rmse_one_set, columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Yxs', 'm_s', 'rmse'])
    df_all_sets = pd.concat([df_all_sets,df_1set], axis=0, ignore_index=True)


df_all_sets.sort_values(by=['rmse'], ascending=True, inplace=True)
df_all_sets.to_csv(f'data/estimation/2906_1/data_2906.csv')

df_all_sets.head(10)